In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
import joblib

In [19]:
import joblib
results = joblib.load('triology.joblib')
docs_texts = joblib.load('triology_docs.joblib')

In [20]:
# # Initialize all_texts with leaf_texts
leaf_texts = docs_texts
all_texts = leaf_texts.copy()

# Iterate through the results to extract summaries from each level and add them to all_texts
for level in sorted(results.keys()):
    # Extract summaries from the current level's DataFrame
    summaries = results[level][1]["summaries"].tolist()
    # Extend all_texts with the summaries from the current level
    all_texts.extend(summaries)

In [36]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(host='172.19.14.186',port=19530)

collection_name = "triology1"

In [37]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=768,
    metric_type="IP",  # Inner product distance
    consistency_level="Strong",  # Strong consistency level
)

In [43]:
embedings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004").embed_documents(all_texts)

In [44]:
from tqdm import tqdm

data = []

for i, line in enumerate(tqdm(all_texts, desc="Creating embeddings")):
    data.append({"id": i, "vector": embedings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 1076/1076 [00:00<00:00, 182449.51it/s]

{'insert_count': 1076,
 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215

In [45]:
question = "Who is sita?"


In [46]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=[
        GoogleGenerativeAIEmbeddings(model="models/text-embedding-004").embed_query(question)
    ],  # Use the `emb_text` function to convert the question to an embedding vector
    limit=3,  # Return top 3 results
    search_params={"metric_type": "IP", "params": {}},  # Inner product distance
    output_fields=["text"],  # Return the text field
)

In [47]:
search_res

data: ['[{\'id\': 1034, \'distance\': 0.6961197853088379, \'entity\': {\'text\': "The provided text is an excerpt from a novel, likely focusing on the life of Sita, a prominent figure in Hindu mythology. While the specific novel is not named, we can glean several key pieces of information:\\n\\n**Characters and Relationships:**\\n\\n* **Sita:** The protagonist, a young girl at the start of the excerpt, dealing with the death of her mother, Sunaina. \\n* **Sunaina:** Sita\'s mother, recently deceased. She comes across as strong-willed, wise, and deeply caring for her daughters. \\n* **Janak:** Sita\'s father, the king of Mithila. He appears absorbed in his philosophical pursuits, even in the face of personal tragedy.\\n* **Urmila:** Sita\'s younger sister, four years old and sickly. \\n* **Kushadhwaj:**  Janak\'s younger brother, who dotes on Sita.\\n* **Samichi and Radhika:** Sita\'s friends, offering comfort and support during her grief.\\n\\n**Setting:**\\n\\n* The story unfolds in a

In [48]:
import json

In [49]:
retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "The provided text is an excerpt from a novel, likely focusing on the life of Sita, a prominent figure in Hindu mythology. While the specific novel is not named, we can glean several key pieces of information:\n\n**Characters and Relationships:**\n\n* **Sita:** The protagonist, a young girl at the start of the excerpt, dealing with the death of her mother, Sunaina. \n* **Sunaina:** Sita's mother, recently deceased. She comes across as strong-willed, wise, and deeply caring for her daughters. \n* **Janak:** Sita's father, the king of Mithila. He appears absorbed in his philosophical pursuits, even in the face of personal tragedy.\n* **Urmila:** Sita's younger sister, four years old and sickly. \n* **Kushadhwaj:**  Janak's younger brother, who dotes on Sita.\n* **Samichi and Radhika:** Sita's friends, offering comfort and support during her grief.\n\n**Setting:**\n\n* The story unfolds in ancient India, primarily in and around Mithila, a kingdom ruled by Sita's father.\n\

In [50]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [53]:
context

"The provided text is an excerpt from a novel, likely focusing on the life of Sita, a prominent figure in Hindu mythology. While the specific novel is not named, we can glean several key pieces of information:\n\n**Characters and Relationships:**\n\n* **Sita:** The protagonist, a young girl at the start of the excerpt, dealing with the death of her mother, Sunaina. \n* **Sunaina:** Sita's mother, recently deceased. She comes across as strong-willed, wise, and deeply caring for her daughters. \n* **Janak:** Sita's father, the king of Mithila. He appears absorbed in his philosophical pursuits, even in the face of personal tragedy.\n* **Urmila:** Sita's younger sister, four years old and sickly. \n* **Kushadhwaj:**  Janak's younger brother, who dotes on Sita.\n* **Samichi and Radhika:** Sita's friends, offering comfort and support during her grief.\n\n**Setting:**\n\n* The story unfolds in ancient India, primarily in and around Mithila, a kingdom ruled by Sita's father.\n\n**Plot Summary:

In [56]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Pull the prompt from LangChain Hub
prompt = hub.pull("rlm/rag-prompt")

# Post-processing function
def format_docs(context):
    return context

# Initialize the language model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-001")

# Chain definition
rag_chain = (
    {"context": RunnablePassthrough() | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "Who is sita?"

# Invoke the chain with the context and question
response = rag_chain.invoke({"context": context, "question": question})

# Print the response
print(response)

Sita is the protagonist of the novel excerpt. She is a young girl who is dealing with the death of her mother, Sunaina, and has a mysterious past surrounding her birth and adoption. 

